In [58]:
import numpy as np
import pandas as pd
import tqdm
from sklearn import tree
from kmodes.kmodes import KModes
from sklearn.cluster import SpectralClustering
from collections import Counter
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.cluster import AffinityPropagation
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
import math 
import sklearn
import scipy.special as sp
from tqdm import tqdm

from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import FastICA
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
%matplotlib notebook

df=[]
df1=[]
for i in range(7):
    df.append(pd.read_csv("/home/franc/franc/matches/b"+str(i)+".csv"))

#for i in range(7):
#    df.append(pd.read_csv("/home/franc/franc/matches/n"+str(i)+".csv"))
    
#for i in range(7):
#    df.append(pd.read_csv("/home/franc/franc/matches/d2"+str(i)+".csv"))
    
#for i in range(6):
#    df.append(pd.read_csv("/home/franc/franc/matches/d1"+str(i)+".csv"))    
#df1.append(pd.read_csv("/home/franc/franc/matches/DNK.csv"))

df=pd.concat(df,sort=False)
#df1=pd.concat(df1,sort=False)

In [59]:
def Probability(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * ((rating1) - rating2) / 400)) 
  
  
# Function to calculate Elo rating 
# K is a constant. 
# d determines whether 
# Player A wins or Player B.  
def EloRating(Ra, Rb, K, d): 
   
      
    # To calculate the Winning 
    # Probability of Player B 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pa = nprob(Ra, Rb) 
    Pb = 1-Pa
    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if (d == 1) : 
        Ra = Ra + K * (1 - Pa) 
        Rb = Rb + K * (0 - Pb) 
      
  
    # Case -2 When Player B wins 
    # Updating the Elo Ratings 
    elif (d==0.5) : 
        Ra = Ra + K * (0.5 - Pa) 
        Rb = Rb + K * (0.5 - Pb) 
      
    else : 
        Ra = Ra + K * (0 - Pa) 
        Rb = Rb + K * (1 - Pb) 
      
    return Ra,Rb
    print("Updated Ratings:-") 
    print("Ra =", round(Ra, 6)," Rb =", round(Rb, 6))

def Probabilityad(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * ((rating1) - rating2) / 400)) 

def nprob(rating1, rating2): 
  
    return 1/2*(1 + sp.erf((rating1-rating2)/(300*math.sqrt(2)))) 

def spreadelo(Ra, Rb, K, ga,gb,n): 
   
      
    # To calculate the Winning 
    # Probability of Player B 
    Pa = nprob(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pb = 1-Pa 
  
    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if ga-gb > n:
        Ra=Ra+K*(1-Pa)
        Rb=Rb+K*(0-Pb)
    else:
        Ra=Ra+K*(0-Pa)
        Rb=Rb+K*(1-Pb)
      
    return Ra,Rb

def adelo(Ra, Rb, K, g,n): 
   
      
    # To calculate the Winning 
    # Probability of Player B 
    Pa = nprob(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pb = 1-Pa 
  
    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if g > n:
        Ra=Ra+K*(1-Pa)
        Rb=Rb+K*(0-Pb)
    else:
        Ra=Ra+K*(0-Pa)
        Rb=Rb+K*(1-Pb)
      
    return Ra,Rb

def kgvelo(Ra, Rb, K, ga,gb): 
   
      
    # To calculate the Winning 
    # Probability of Player B 
    Pa = nprob(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pb = 1-Pa 
  
    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if ga==0:
        Ra=Ra+K*(0-Pa)
    else:
        Ra=Ra+K*(1-Pa)
        
    if gb==0:
        Rb=Rb+K*(0-Pb)
    else:
        Rb=Rb+K*(1-Pb)
      
    return Ra,Rb

In [120]:
rat_dic={}
fet_set=[]
tar_set=[]

In [121]:
df.HomeTeam.unique()

array(['Bergen', 'Club Brugge', 'Germinal', 'Kortrijk', 'Mechelen',
       'Genk', 'Gent', 'Standard', 'Lierse', 'Anderlecht',
       'Cercle Brugge', 'Charleroi', 'Lokeren', 'Waasland-Beveren',
       'Oud-Heverlee Leuven', 'Waregem', 'Oostende', 'Westerlo',
       'Mouscron-Peruwelz', 'St Truiden', 'Mouscron', 'Eupen', 'Antwerp'],
      dtype=object)

In [122]:
df['Date']=pd.to_datetime(df['Date']
                              ,infer_datetime_format=True)
last_two=df[(df["Date"].dt.year>2012) & (df["Date"].dt.year<2017)]
for i in df.HomeTeam.unique():
    rat_dic[i]={"tr":1500,"tr1":1500,"tr-1":1500,"tr2":1500,"tr-2":1500,"ar":1500,"ar1":1500,"ar2":1500,"dr":1500,"dr1":1500,"dr2":1500,"kgv":1500,"nm":0}

test=df[(df["Date"].dt.year>=2014)]

for i in range(len(last_two)):
    rat_dic[last_two.iloc[i,2]]["tr"],rat_dic[last_two.iloc[i,3]]["tr"]=spreadelo(rat_dic[last_two.iloc[i,2]]["tr"],rat_dic[last_two.iloc[i,3]]["tr"],60,last_two.iloc[i,4],last_two.iloc[i,5],0)
    rat_dic[last_two.iloc[i,2]]["tr1"],rat_dic[last_two.iloc[i,3]]["tr-1"]=spreadelo(rat_dic[last_two.iloc[i,2]]["tr1"],rat_dic[last_two.iloc[i,3]]["tr-1"],60,last_two.iloc[i,4],last_two.iloc[i,5],1)
    rat_dic[last_two.iloc[i,2]]["tr2"],rat_dic[last_two.iloc[i,3]]["tr-2"]=spreadelo(rat_dic[last_two.iloc[i,2]]["tr2"],rat_dic[last_two.iloc[i,3]]["tr-2"],60,last_two.iloc[i,4],last_two.iloc[i,5],2)
    rat_dic[last_two.iloc[i,3]]["tr1"],rat_dic[last_two.iloc[i,2]]["tr-1"]=spreadelo(rat_dic[last_two.iloc[i,3]]["tr1"],rat_dic[last_two.iloc[i,2]]["tr-1"],60,last_two.iloc[i,5],last_two.iloc[i,4],1)
    rat_dic[last_two.iloc[i,3]]["tr2"],rat_dic[last_two.iloc[i,2]]["tr-2"]=spreadelo(rat_dic[last_two.iloc[i,3]]["tr2"],rat_dic[last_two.iloc[i,2]]["tr-2"],60,last_two.iloc[i,5],last_two.iloc[i,4],2)
    rat_dic[last_two.iloc[i,2]]["ar"],rat_dic[last_two.iloc[i,3]]["dr"]=adelo(rat_dic[last_two.iloc[i,2]]["ar"],rat_dic[last_two.iloc[i,3]]["dr"],60,last_two.iloc[i,4],0)
    rat_dic[last_two.iloc[i,3]]["ar"],rat_dic[last_two.iloc[i,2]]["dr"]=adelo(rat_dic[last_two.iloc[i,3]]["ar"],rat_dic[last_two.iloc[i,2]]["dr"],60,last_two.iloc[i,5],0)
    rat_dic[last_two.iloc[i,2]]["ar1"],rat_dic[last_two.iloc[i,3]]["dr1"]=adelo(rat_dic[last_two.iloc[i,2]]["ar1"],rat_dic[last_two.iloc[i,3]]["dr1"],60,last_two.iloc[i,4],1)
    rat_dic[last_two.iloc[i,3]]["ar1"],rat_dic[last_two.iloc[i,2]]["dr1"]=adelo(rat_dic[last_two.iloc[i,3]]["ar1"],rat_dic[last_two.iloc[i,2]]["dr1"],60,last_two.iloc[i,5],1)    
    rat_dic[last_two.iloc[i,2]]["ar2"],rat_dic[last_two.iloc[i,3]]["dr2"]=adelo(rat_dic[last_two.iloc[i,2]]["ar2"],rat_dic[last_two.iloc[i,3]]["dr2"],60,last_two.iloc[i,4],2)
    rat_dic[last_two.iloc[i,3]]["ar2"],rat_dic[last_two.iloc[i,2]]["dr2"]=adelo(rat_dic[last_two.iloc[i,3]]["ar2"],rat_dic[last_two.iloc[i,2]]["dr2"],60,last_two.iloc[i,5],2)     
    rat_dic[last_two.iloc[i,2]]["kgv"],rat_dic[last_two.iloc[i,3]]["kgv"]=kgvelo(rat_dic[last_two.iloc[i,2]]["kgv"],rat_dic[last_two.iloc[i,3]]["kgv"],60,last_two.iloc[i,4],last_two.iloc[i,5])
    rat_dic[last_two.iloc[i,2]]["nm"]+=1
    rat_dic[last_two.iloc[i,3]]["nm"]+=1
    

#tar_set ga,gb,tg,s,kgv,oo,to,tho
for t in range(len(test)):
    tmp=[]
    tmp1=[]
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["tr"],rat_dic[test.iloc[t,3]]["tr"]))   
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["tr1"],rat_dic[test.iloc[t,3]]["tr-1"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["tr2"],rat_dic[test.iloc[t,3]]["tr-2"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["tr1"],rat_dic[test.iloc[t,2]]["tr-1"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["tr2"],rat_dic[test.iloc[t,2]]["tr-2"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["ar"],rat_dic[test.iloc[t,3]]["dr"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["ar"],rat_dic[test.iloc[t,2]]["dr"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["ar1"],rat_dic[test.iloc[t,3]]["dr1"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["ar1"],rat_dic[test.iloc[t,2]]["dr1"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["ar2"],rat_dic[test.iloc[t,3]]["dr2"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["ar2"],rat_dic[test.iloc[t,2]]["dr2"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["kgv"],rat_dic[test.iloc[t,3]]["kgv"]))
    tmp1.append(test.iloc[t,4])
    tmp1.append(test.iloc[t,5])
    tmp1.append(test.iloc[t,4]+test.iloc[t,5])
    tmp1.append(test.iloc[t,4]-test.iloc[t,5])
    tmp1.append((test.iloc[t,4]!=0) and (test.iloc[t,5]!=0))
    tmp1.append(1*((test.iloc[t,4]+test.iloc[t,5])>1.5))
    tmp1.append(1*((test.iloc[t,4]+test.iloc[t,5])>2.5))
    tmp1.append(1*((test.iloc[t,4]+test.iloc[t,5])>3.5))
    fet_set.append(np.array(tmp))
    tar_set.append(np.array(tmp1))

In [79]:
df1['Date']=pd.to_datetime(df1['Date']
                              ,infer_datetime_format=True)
last_two1=df1[(df1["Date"].dt.year>2012) & (df1["Date"].dt.year<2017)]
for i in df1.Home.unique():
    rat_dic[i]={"tr":1500,"tr1":1500,"tr-1":1500,"tr2":1500,"tr-2":1500,"ar":1500,"ar1":1500,"ar2":1500,"dr":1500,"dr1":1500,"dr2":1500,"kgv":1500,"nm":0}

test1=df1[(df1["Date"].dt.year>=2014)]

for i in range(len(last_two1)):
    rat_dic[last_two1.iloc[i,5]]["tr"],rat_dic[last_two1.iloc[i,6]]["tr"]=spreadelo(rat_dic[last_two1.iloc[i,5]]["tr"],rat_dic[last_two1.iloc[i,6]]["tr"],60,last_two1.iloc[i,7],last_two1.iloc[i,8],0)
    rat_dic[last_two1.iloc[i,5]]["tr1"],rat_dic[last_two1.iloc[i,6]]["tr-1"]=spreadelo(rat_dic[last_two1.iloc[i,5]]["tr1"],rat_dic[last_two1.iloc[i,6]]["tr-1"],60,last_two1.iloc[i,7],last_two1.iloc[i,8],1)
    rat_dic[last_two1.iloc[i,5]]["tr2"],rat_dic[last_two1.iloc[i,6]]["tr-2"]=spreadelo(rat_dic[last_two1.iloc[i,5]]["tr2"],rat_dic[last_two1.iloc[i,6]]["tr-2"],60,last_two1.iloc[i,7],last_two1.iloc[i,8],2)
    rat_dic[last_two1.iloc[i,6]]["tr1"],rat_dic[last_two1.iloc[i,5]]["tr-1"]=spreadelo(rat_dic[last_two1.iloc[i,6]]["tr1"],rat_dic[last_two1.iloc[i,5]]["tr-1"],60,last_two1.iloc[i,8],last_two1.iloc[i,7],1)
    rat_dic[last_two1.iloc[i,6]]["tr2"],rat_dic[last_two1.iloc[i,5]]["tr-2"]=spreadelo(rat_dic[last_two1.iloc[i,6]]["tr2"],rat_dic[last_two1.iloc[i,5]]["tr-2"],60,last_two1.iloc[i,8],last_two1.iloc[i,7],2)
    rat_dic[last_two1.iloc[i,5]]["ar"],rat_dic[last_two1.iloc[i,6]]["dr"]=adelo(rat_dic[last_two1.iloc[i,5]]["ar"],rat_dic[last_two1.iloc[i,6]]["dr"],60,last_two1.iloc[i,7],0)
    rat_dic[last_two1.iloc[i,6]]["ar"],rat_dic[last_two1.iloc[i,5]]["dr"]=adelo(rat_dic[last_two1.iloc[i,6]]["ar"],rat_dic[last_two1.iloc[i,5]]["dr"],60,last_two1.iloc[i,8],0)
    rat_dic[last_two1.iloc[i,5]]["ar1"],rat_dic[last_two1.iloc[i,6]]["dr1"]=adelo(rat_dic[last_two1.iloc[i,5]]["ar1"],rat_dic[last_two1.iloc[i,6]]["dr1"],60,last_two1.iloc[i,7],1)
    rat_dic[last_two1.iloc[i,6]]["ar1"],rat_dic[last_two1.iloc[i,5]]["dr1"]=adelo(rat_dic[last_two1.iloc[i,6]]["ar1"],rat_dic[last_two1.iloc[i,5]]["dr1"],60,last_two1.iloc[i,8],1)    
    rat_dic[last_two1.iloc[i,5]]["ar2"],rat_dic[last_two1.iloc[i,6]]["dr2"]=adelo(rat_dic[last_two1.iloc[i,5]]["ar2"],rat_dic[last_two1.iloc[i,6]]["dr2"],60,last_two1.iloc[i,7],2)
    rat_dic[last_two1.iloc[i,6]]["ar2"],rat_dic[last_two1.iloc[i,5]]["dr2"]=adelo(rat_dic[last_two1.iloc[i,6]]["ar2"],rat_dic[last_two1.iloc[i,5]]["dr2"],60,last_two1.iloc[i,8],2)     
    rat_dic[last_two1.iloc[i,5]]["kgv"],rat_dic[last_two1.iloc[i,6]]["kgv"]=kgvelo(rat_dic[last_two1.iloc[i,5]]["kgv"],rat_dic[last_two1.iloc[i,6]]["kgv"],60,last_two1.iloc[i,7],last_two1.iloc[i,8])
    rat_dic[last_two1.iloc[i,5]]["nm"]+=1
    rat_dic[last_two1.iloc[i,6]]["nm"]+=1
    
#tar_set ga,gb,tg,s,kgv,oo,to,tho
for t in range(len(test1)):
    tmp=[]
    tmp1=[]
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["tr"],rat_dic[test1.iloc[t,6]]["tr"]))   
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["tr1"],rat_dic[test1.iloc[t,6]]["tr-1"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["tr2"],rat_dic[test1.iloc[t,6]]["tr-2"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["tr1"],rat_dic[test1.iloc[t,5]]["tr-1"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["tr2"],rat_dic[test1.iloc[t,5]]["tr-2"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["ar"],rat_dic[test1.iloc[t,6]]["dr"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["ar"],rat_dic[test1.iloc[t,5]]["dr"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["ar1"],rat_dic[test1.iloc[t,6]]["dr1"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["ar1"],rat_dic[test1.iloc[t,5]]["dr1"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["ar2"],rat_dic[test1.iloc[t,6]]["dr2"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["ar2"],rat_dic[test1.iloc[t,5]]["dr2"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["kgv"],rat_dic[test1.iloc[t,6]]["kgv"]))
    tmp1.append(test1.iloc[t,7])
    tmp1.append(test1.iloc[t,8])
    tmp1.append(test1.iloc[t,7]+test1.iloc[t,8])
    tmp1.append(test1.iloc[t,7]-test1.iloc[t,8])
    tmp1.append((test1.iloc[t,7]!=0) and (test1.iloc[t,8]!=0))
    tmp1.append(1*((test1.iloc[t,7]+test1.iloc[t,8])>1.5))
    tmp1.append(1*((test1.iloc[t,7]+test1.iloc[t,8])>2.5))
    tmp1.append(1*((test1.iloc[t,7]+test1.iloc[t,8])>3.5))
    fet_set.append(np.array(tmp))
    tar_set.append(np.array(tmp1))



TypeError: list indices must be integers or slices, not str

In [123]:
n=1
tmp=[]
t1="Anderlecht"
t2="Oostende"
tmp.append(nprob(rat_dic[t1]["tr"],rat_dic[t2]["tr"]))   
tmp.append(nprob(rat_dic[t1]["tr1"],rat_dic[t2]["tr-1"]))
tmp.append(nprob(rat_dic[t1]["tr2"],rat_dic[t2]["tr-2"]))
tmp.append(nprob(rat_dic[t2]["tr1"],rat_dic[t1]["tr-1"]))
tmp.append(nprob(rat_dic[t2]["tr2"],rat_dic[t1]["tr-2"]))
tmp.append(nprob(rat_dic[t1]["ar"],rat_dic[t2]["dr"]))
tmp.append(nprob(rat_dic[t2]["ar"],rat_dic[t1]["dr"]))
tmp.append(nprob(rat_dic[t1]["ar1"],rat_dic[t2]["dr1"]))
tmp.append(nprob(rat_dic[t2]["ar1"],rat_dic[t1]["dr1"]))
tmp.append(nprob(rat_dic[t1]["ar2"],rat_dic[t2]["dr2"]))
tmp.append(nprob(rat_dic[t2]["ar2"],rat_dic[t1]["dr2"]))
tmp.append(nprob(rat_dic[t1]["kgv"],rat_dic[t2]["kgv"]))
fet_set.append(tmp)

In [124]:
fet_set=np.array(fet_set)
tar_set=np.array(tar_set)
nc=200
f=[[[[] for i in range(nc)] for j in range(3)]for i in range(4)]
ft=[]

In [125]:
rft=[]
d1=[]
d2=[]
d=[]
d3=[np.array([0,0,0,0,0,0,0,1])*n]
for j in tqdm(range(1,2)):
    ft=[]
    f=[[[[] for i in range(nc)] for j in range(3)]for i in range(4)]
    pca = PCA(n_components=3)
    l=pca.fit_transform(fet_set[:,:12])
    kmeans = KMeans(n_clusters=nc, random_state=0).fit(l)
    clustering = AgglomerativeClustering(n_clusters=nc,linkage="ward").fit(l)
    clustering1 = AgglomerativeClustering(n_clusters=nc,linkage="complete").fit(l)

    for i in range(len(kmeans.labels_)-1):
        f[0][0][kmeans.labels_[i]].append(tar_set[i])
    for i in range(len(clustering.labels_)-1):
        f[0][1][clustering.labels_[i]].append(tar_set[i])
    for i in range(len(clustering1.labels_)-1):
        f[0][2][clustering1.labels_[i]].append(tar_set[i])
    
    ft+=[kmeans.labels_[-1*j],clustering.labels_[-1*j],clustering1.labels_[-1*j]]

    transformer = KernelPCA(n_components=3, kernel="cosine")
    l=transformer.fit_transform(fet_set[:,:12])
    kmeans = KMeans(n_clusters=nc, random_state=0).fit(l)
    clustering = AgglomerativeClustering(n_clusters=nc,linkage="ward").fit(l)
    clustering1 = AgglomerativeClustering(n_clusters=nc,linkage="complete").fit(l)

    for i in range(len(kmeans.labels_)-1):
        f[1][0][kmeans.labels_[i]].append(tar_set[i])
    for i in range(len(clustering.labels_)-1):
        f[1][1][clustering.labels_[i]].append(tar_set[i])
    for i in range(len(clustering1.labels_)-1):
        f[1][2][clustering1.labels_[i]].append(tar_set[i])
    
    ft+=[kmeans.labels_[-1*j],clustering.labels_[-1*j],clustering1.labels_[-1*j]]

    transformer = KernelPCA(n_components=4, kernel="rbf")
    l=transformer.fit_transform(fet_set[:,:12])
    kmeans = KMeans(n_clusters=nc, random_state=0).fit(l)
    clustering = AgglomerativeClustering(n_clusters=nc,linkage="ward").fit(l)
    clustering1 = AgglomerativeClustering(n_clusters=nc,linkage="complete").fit(l)

    for i in range(len(kmeans.labels_)-1):
        f[2][0][kmeans.labels_[i]].append(tar_set[i])
    for i in range(len(clustering.labels_)-1):
        f[2][1][clustering.labels_[i]].append(tar_set[i])
    for i in range(len(clustering1.labels_)-1):
        f[2][2][clustering1.labels_[i]].append(tar_set[i])
    
    ft+=[kmeans.labels_[-1*j],clustering.labels_[-1*j],clustering1.labels_[-1*j]]

    transformer = FastICA(n_components=4,random_state=0,tol=1)
    l=transformer.fit_transform(fet_set[:,:12])
    kmeans = KMeans(n_clusters=nc, random_state=0).fit(l)
    clustering = AgglomerativeClustering(n_clusters=nc,linkage="ward").fit(l)
    clustering1 = AgglomerativeClustering(n_clusters=nc,linkage="complete").fit(l)

    for i in range(len(kmeans.labels_)-1):
        f[3][0][kmeans.labels_[i]].append(tar_set[i])
    for i in range(len(clustering.labels_)-1):
        f[3][1][clustering.labels_[i]].append(tar_set[i])
    for i in range(len(clustering1.labels_)-1):
        f[3][2][clustering1.labels_[i]].append(tar_set[i])
    
    ft+=[kmeans.labels_[-1*j],clustering.labels_[-1*j],clustering1.labels_[-1*j]]
    d3+=[tar_set[len(tar_set)-j-1]]
    tm2=[]
    tm1=[]
    tm=[]
    for k in range(len(ft)):
        #print(int(k/3),(k-3*int(k/3))%3,ft[k],np.array(f[int(k/3)][(k-3*int(k/3))%3][ft[k]]))
        tm+=[np.array([np.array(f[int(k/3)][(k-3*int(k/3))%3][ft[k]]) ])]
        tm1+=[np.array([np.array(f[int(k/3)][(k-3*int(k/3))%3][ft[k]])[:,u].mean() for u in range(len(tar_set[0]))])]
        tm2+=[len(f[int(k/3)][(k-3*int(k/3))%3][ft[k]])]
        #print("-------------------------------------------------")
    d+=[tm]
    d1+=[tm1]
    d2+=[tm2]



100%|██████████| 1/1 [00:02<00:00,  2.91s/it]


In [126]:
for i in range(len(d2)):
    for j in range(len(d2[i])):
        d1[i][j]*=(d2[i][j]/sum(d2[i]))
for i in range(len(d1)):
    d1[i]=np.transpose(d1[i])

In [127]:
ff=[]
ki=0
g=-10
transformer = KernelPCA(n_components=4, kernel="rbf")
l=transformer.fit_transform(fet_set[:,:8])
pw=sklearn.metrics.pairwise.cosine_similarity(l)
ff.append(list(map(list,\
                   list(tar_set[pw[-1-ki].argsort()[g:][::-1][1:]]))))
pw=sklearn.metrics.pairwise.laplacian_kernel(l)
ff.append(list(map(list,\
                   list(tar_set[pw[-1-ki].argsort()[g:][::-1][1:]]))))
for i in range(12):
    ff.append([[d[ki][i][0][u][p] for p in range(len(d[ki][i][0][u]))] for u in range(len(d[ki][i][0]))])
dic={}
dic1={}
for i in range(len(ff)):
    for j in range(len(ff[i])):
        if(str(ff[i][j]) not in dic):
            dic[str(ff[i][j])]=ff[i][j]
        if(str(ff[i][j]) not in dic1):
            dic1[str(ff[i][j])]=1
        else:
            dic1[str(ff[i][j])]+=1
print(len(ff),dic1)            
[dic[i] for i in [k for (k,v) in dic1.items() if v >= len(ff)]],d3[ki]

14 {'[4, 0, 4, 4, 0, 1, 1, 1]': 18, '[5, 2, 7, 3, 1, 1, 1, 1]': 14, '[1, 1, 2, 0, 1, 1, 0, 0]': 31, '[1, 0, 1, 1, 0, 0, 0, 0]': 20, '[3, 0, 3, 3, 0, 1, 1, 0]': 20, '[2, 0, 2, 2, 0, 1, 0, 0]': 2, '[2, 1, 3, 1, 1, 1, 1, 0]': 2, '[7, 1, 8, 6, 1, 1, 1, 1]': 3, '[2, 2, 4, 0, 1, 1, 1, 1]': 6, '[0, 0, 0, 0, 0, 0, 0, 0]': 3, '[1, 5, 6, -4, 1, 1, 1, 1]': 2}


([[4, 0, 4, 4, 0, 1, 1, 1],
  [5, 2, 7, 3, 1, 1, 1, 1],
  [1, 1, 2, 0, 1, 1, 0, 0],
  [1, 0, 1, 1, 0, 0, 0, 0],
  [3, 0, 3, 3, 0, 1, 1, 0]],
 array([0, 0, 0, 0, 0, 0, 0, 1]))